In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# to open fits files
from astropy.table import Table

In [ ]:
source_list = Table.read("parent_sample/EELR_HSCmag_from_SDSSspec.fits")

In [ ]:
z = source_list[:]["Z"]
griz = np.nan_to_num(source_list[:]["MAG_AB_LINEONLY"], nan=30.0)[:, :4]

mask = np.all((0 < griz) & (griz < 28.5), axis=1)
z = z[mask]
griz = griz[mask]

print(z.shape, griz.shape)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

kernel = 1.0 * RBF() + WhiteKernel(noise_level_bounds=(1e-5, 1.5))

In [ ]:
z_ = np.linspace(z.min(), z.max(), 100)

In [ ]:
bands = ["g", "r", "i", "z"]
models = {}
fig, axs = plt.subplots(2, 3, figsize=(15, 8))

r, c = 0, 0
for b1 in range(4):
    for b2 in range(b1+1, 4):
        color = griz[:, b1] - griz[:, b2]
        gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0, normalize_y=True)
        gp.fit(z.reshape((-1, 1)), color)
        models[f"{bands[b1]}-{bands[b2]}"] = gp
        print(gp.kernel_)
        mean, std = gp.predict(z_.reshape((-1, 1)), return_std=True)
        
        axs[r, c].plot(z_, mean)
        axs[r, c].fill_between(z_, mean - std, mean + std, alpha=0.5)
        axs[r, c].scatter(z, color)
        axs[r, c].set_xlabel("z")
        axs[r, c].set_ylabel(f"{bands[b1]} - {bands[b2]}")
        c += 1
        if c == 3:
            r += 1
            c = 0

plt.savefig(f"griz_vs_z.png", dpi=200, bbox_inches="tight")
plt.show()

In [ ]:
import pickle

with open("gp_models.pickle", "wb") as outfile:
    pickle.dump(models, outfile)

In [ ]:
gy_mean = np.ones(z_.shape) * (25 - (26 + 18) / 2)

plt.figure(figsize=(5, 4))
plt.plot(z_, gy_mean)
plt.fill_between(z_, gy_mean - 4, gy_mean + 4, alpha=0.5)
plt.xlabel("z")
plt.ylabel("g - y")
plt.savefig("gy_vs_z.png", dpi=200, bbox_inches="tight")
plt.show()